In [12]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

# ✅ إعداد الاتصال بـ SQL Server
server = 'localhost'
database = 'Retail_Inventory_DB'
driver = 'ODBC Driver 17 for SQL Server'

connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes;"
params = urllib.parse.quote_plus(connection_string)

# ✅ إنشاء المحرك
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# ==============================
# ✅ تحميل جدول المنتجات (Products)
# ==============================

try:
    df_products = pd.read_csv(r'C:\Users\Hanin Baher\OneDrive\Documents\Retail\datasets\products.csv')  # استخدم r'' لتجنب مشاكل الـ backslash
    df_products.rename(columns={'Product ID': 'Product_ID'}, inplace=True)
    
    df_products.to_sql('Products', con=engine, if_exists='append', index=False)
    print("✅ Products table loaded successfully.")
except Exception as e:
    print("❌ Error loading Products:", e)

# ==============================
# ✅ تحميل جدول المتاجر (Stores)
# ==============================

try:
    df_stores = pd.read_csv(r'C:\Users\Hanin Baher\OneDrive\Documents\Retail\datasets\store.csv')
    df_stores.rename(columns={'Store ID': 'Store_ID'}, inplace=True)

    df_stores.to_sql('Stores', con=engine, if_exists='append', index=False)
    print("✅ Stores table loaded successfully.")
except Exception as e:
    print("❌ Error loading Stores:", e)


✅ Products table loaded successfully.
✅ Stores table loaded successfully.


In [14]:
df_dates = pd.read_csv(r'C:\Users\Hanin Baher\OneDrive\Documents\Retail\datasets\dates.csv')   
df_dates.rename(columns={'Holiday/Promotion': 'Holiday_Promotion'}, inplace=True)

try:
    df_dates.to_sql('Dates', con=engine, if_exists='append', index=False)
    print("✅ Dates table loaded successfully.")
except Exception as e:
    print("❌ Error loading dates:", e)

✅ Dates table loaded successfully.


In [15]:
df_sales = pd.read_csv(r'C:\Users\Hanin Baher\OneDrive\Documents\Retail\datasets\sales.csv')

# ✅ Rename columns to match DB table
df_sales.rename(columns={
    'Store ID': 'Store_ID',
    'Product ID': 'Product_ID',
    'Units Sold': 'Units_Sold',
    'Units Ordered': 'Units_Ordered'
}, inplace=True)

# ✅ Load mapping from Stores table
store_map = pd.read_sql('SELECT Store_Region_ID, Store_ID FROM Stores', con=engine)
store_dict = dict(zip(store_map['Store_ID'], store_map['Store_Region_ID']))

# ✅ Load mapping from Products table
product_map = pd.read_sql('SELECT Product_Entry_ID, Product_ID FROM Products', con=engine)
product_dict = dict(zip(product_map['Product_ID'], product_map['Product_Entry_ID']))

# ✅ Map string IDs to surrogate keys
df_sales['Store_ID'] = df_sales['Store_ID'].map(store_dict)
df_sales['Product_ID'] = df_sales['Product_ID'].map(product_dict)

# ✅ Check for unmapped rows
missing_store_ids = df_sales[df_sales['Store_ID'].isna()]
missing_product_ids = df_sales[df_sales['Product_ID'].isna()]

if not missing_store_ids.empty or not missing_product_ids.empty:
    print("❌ There are unmapped IDs:")
    if not missing_store_ids.empty:
        print("Missing Store_IDs:\n", missing_store_ids[['Date', 'Store_ID']].drop_duplicates())
    if not missing_product_ids.empty:
        print("Missing Product_IDs:\n", missing_product_ids[['Date', 'Product_ID']].drop_duplicates())
else:
    # ✅ Load to SQL if no missing keys
    try:
        df_sales.to_sql('Sales', con=engine, if_exists='append', index=False)
        print("✅ Sales table loaded successfully.")
    except Exception as e:
        print("❌ Error loading Sales:", e)

✅ Sales table loaded successfully.


In [16]:
# ✅ قراءة بيانات العروض
df_promotions = pd.read_csv(r'C:\Users\Hanin Baher\OneDrive\Documents\Retail\datasets\promotion.csv')

# ✅ تعديل أسماء الأعمدة لتتناسب مع قاعدة البيانات
df_promotions.rename(columns={
    'Store ID': 'Store_ID',
    'Product ID': 'Product_ID',
    'Holiday/Promotion': 'Holiday_Promotion'
}, inplace=True)

# ✅ تحميل المفاتيح البديلة من قاعدة البيانات
store_map = pd.read_sql('SELECT Store_Region_ID, Store_ID FROM Stores', con=engine)
store_dict = dict(zip(store_map['Store_ID'], store_map['Store_Region_ID']))

product_map = pd.read_sql('SELECT Product_Entry_ID, Product_ID FROM Products', con=engine)
product_dict = dict(zip(product_map['Product_ID'], product_map['Product_Entry_ID']))

# ✅ تحويل الـ Store_ID و Product_ID إلى المفاتيح البديلة
df_promotions['Store_ID'] = df_promotions['Store_ID'].map(store_dict)
df_promotions['Product_ID'] = df_promotions['Product_ID'].map(product_dict)

# ✅ التحقق من القيم المفقودة (IDs غير مرتبطة)
missing_store_ids = df_promotions[df_promotions['Store_ID'].isna()]
missing_product_ids = df_promotions[df_promotions['Product_ID'].isna()]

if not missing_store_ids.empty or not missing_product_ids.empty:
    print("❌ There are unmapped IDs in Promotions:")
    if not missing_store_ids.empty:
        print("Missing Store_IDs:\n", missing_store_ids[['Date', 'Store_ID']].drop_duplicates())
    if not missing_product_ids.empty:
        print("Missing Product_IDs:\n", missing_product_ids[['Date', 'Product_ID']].drop_duplicates())
else:
    # ✅ تحميل البيانات إلى قاعدة البيانات
    try:
        df_promotions.to_sql('Promotions', con=engine, if_exists='append', index=False)
        print("✅ Promotions table loaded successfully.")
    except Exception as e:
        print("❌ Error loading Promotions:", e)


✅ Promotions table loaded successfully.


In [17]:
# ✅ قراءة بيانات الجرد (Inventory)
df_inventory = pd.read_csv(r'C:\Users\Hanin Baher\OneDrive\Documents\Retail\datasets\inventory.csv')

# ✅ تعديل أسماء الأعمدة لتتناسب مع قاعدة البيانات
df_inventory.rename(columns={
    'Store ID': 'Store_ID',
    'Product ID': 'Product_ID',
    'Inventory Level': 'Inventory_Level'
}, inplace=True)

# ✅ تحميل المفاتيح البديلة من قاعدة البيانات
store_map = pd.read_sql('SELECT Store_Region_ID, Store_ID FROM Stores', con=engine)
store_dict = dict(zip(store_map['Store_ID'], store_map['Store_Region_ID']))

product_map = pd.read_sql('SELECT Product_Entry_ID, Product_ID FROM Products', con=engine)
product_dict = dict(zip(product_map['Product_ID'], product_map['Product_Entry_ID']))

# ✅ تحويل Store_ID و Product_ID للمفاتيح البديلة
df_inventory['Store_ID'] = df_inventory['Store_ID'].map(store_dict)
df_inventory['Product_ID'] = df_inventory['Product_ID'].map(product_dict)

# ✅ التحقق من القيم المفقودة
missing_store_ids = df_inventory[df_inventory['Store_ID'].isna()]
missing_product_ids = df_inventory[df_inventory['Product_ID'].isna()]

if not missing_store_ids.empty or not missing_product_ids.empty:
    print("❌ There are unmapped IDs in Inventory:")
    if not missing_store_ids.empty:
        print("Missing Store_IDs:\n", missing_store_ids[['Date', 'Store_ID']].drop_duplicates())
    if not missing_product_ids.empty:
        print("Missing Product_IDs:\n", missing_product_ids[['Date', 'Product_ID']].drop_duplicates())
else:
    # ✅ تحميل البيانات إلى SQL
    try:
        df_inventory.to_sql('Inventory', con=engine, if_exists='append', index=False)
        print("✅ Inventory table loaded successfully.")
    except Exception as e:
        print("❌ Error loading Inventory:", e)


✅ Inventory table loaded successfully.


In [19]:
# ✅ قراءة بيانات التنبؤ بالطلب (Forecast)
df_forecast = pd.read_csv(r'C:\Users\Hanin Baher\OneDrive\Documents\Retail\datasets\forecast.csv')

# ✅ تعديل أسماء الأعمدة لتتناسب مع قاعدة البيانات
df_forecast.rename(columns={
    'Store ID': 'Store_ID',
    'Product ID': 'Product_ID',
    'Demand Forecast': 'Demand_Forecast'
}, inplace=True)

# ✅ تحميل المفاتيح البديلة من SQL
store_map = pd.read_sql('SELECT Store_Region_ID, Store_ID FROM Stores', con=engine)
store_dict = dict(zip(store_map['Store_ID'], store_map['Store_Region_ID']))

product_map = pd.read_sql('SELECT Product_Entry_ID, Product_ID FROM Products', con=engine)
product_dict = dict(zip(product_map['Product_ID'], product_map['Product_Entry_ID']))

# ✅ ربط الـ IDs بالمفاتيح البديلة
df_forecast['Store_ID'] = df_forecast['Store_ID'].map(store_dict)
df_forecast['Product_ID'] = df_forecast['Product_ID'].map(product_dict)

# ✅ التحقق من القيم غير المرتبطة
missing_store_ids = df_forecast[df_forecast['Store_ID'].isna()]
missing_product_ids = df_forecast[df_forecast['Product_ID'].isna()]

if not missing_store_ids.empty or not missing_product_ids.empty:
    print("❌ There are unmapped IDs in Forecast:")
    if not missing_store_ids.empty:
        print("Missing Store_IDs:\n", missing_store_ids[['Date', 'Store_ID']].drop_duplicates())
    if not missing_product_ids.empty:
        print("Missing Product_IDs:\n", missing_product_ids[['Date', 'Product_ID']].drop_duplicates())
else:
    # ✅ تحميل البيانات إلى SQL
    try:
        df_forecast.to_sql('Forecast', con=engine, if_exists='append', index=False)
        print("✅ Forecast table loaded successfully.")
    except Exception as e:
        print("❌ Error loading Forecast:", e)


✅ Forecast table loaded successfully.


In [20]:
import pandas as pd

# تحميل بيانات المنافسين من ملف CSV
df_competitor = pd.read_csv(r'C:\Users\Hanin Baher\OneDrive\Documents\Retail\datasets\competitor.csv')

# تعديل أسماء الأعمدة لتتناسب مع قاعدة البيانات
df_competitor.rename(columns={
    'Product ID': 'Product_ID',
    'Competitor Pricing': 'Competitor_Pricing'
}, inplace=True)

# تحميل المفاتيح البديلة من SQL للمنتجات
product_map = pd.read_sql('SELECT Product_Entry_ID, Product_ID FROM Products', con=engine)
product_dict = dict(zip(product_map['Product_ID'], product_map['Product_Entry_ID']))

# ربط Product_ID بالمفتاح البديل Product_Entry_ID
df_competitor['Product_ID'] = df_competitor['Product_ID'].map(product_dict)

# التحقق من القيم غير المرتبطة
missing_product_ids = df_competitor[df_competitor['Product_ID'].isna()]

if not missing_product_ids.empty:
    print("❌ هناك Product_IDs غير مرتبطة في Competitor:")
    print(missing_product_ids[['Date', 'Product_ID']].drop_duplicates())
else:
    # تحميل البيانات إلى جدول Competitor في قاعدة البيانات
    try:
        df_competitor.to_sql('Competitor', con=engine, if_exists='append', index=False)
        print("✅ تم تحميل بيانات Competitor بنجاح.")
    except Exception as e:
        print("❌ خطأ أثناء تحميل بيانات Competitor:", e)


✅ تم تحميل بيانات Competitor بنجاح.


In [21]:
import pandas as pd

# تحميل بيانات الطقس من ملف CSV
df_weather = pd.read_csv(r'C:\Users\Hanin Baher\OneDrive\Documents\Retail\datasets\weather.csv')

# تعديل أسماء الأعمدة لتتناسب مع قاعدة البيانات
df_weather.rename(columns={
    'Weather Condition': 'Weather_Condition'
}, inplace=True)

# تحميل البيانات إلى جدول Weather في قاعدة البيانات
try:
    df_weather.to_sql('Weather', con=engine, if_exists='append', index=False)
    print("✅ Weather table loaded successfully.")
except Exception as e:
    print("❌ Error loading Weather:", e)


✅ Weather table loaded successfully.
